## Problema del Refugio 117 - Programación Dinámica con SRTBOT

### Diego Silva Madariaga ; Christian Pérez Flores

Tarea ambientada en el universo de Fallout.  
El objetivo es escapar desde la celda (0,0) hasta (n-1,n-1) en una cuadrícula de tamaño n × n, recolectando la mayor cantidad de cápsulas RadAway posibles, en a lo más 2n - 1 pasos y evitando celdas con bombas (B).

Se implementan dos estrategias de programación dinámica usando el esquema SRTBOT:

- **Tabla tridimensional (dp[x][y][t])**
- **Tabla hash (memo[(x, y, t)])**



In [1]:
import time
import random
import tracemalloc
from typing import Dict, Tuple, List
import plotly.graph_objects as go  

#random.seed(42) por si se quiere la misma semilla cada vez que se ejecute

In [2]:
class Refugio117:
    def __init__(self, grid: List[List[str]]):
        self.grid = grid
        self.n = len(grid)
        self.max_steps = 2 * self.n - 1
        self.directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    def valido(self, x: int, y: int) -> bool:
        return 0 <= x < self.n and 0 <= y < self.n and self.grid[x][y] != 'B'

    def radaway(self, x: int, y: int) -> int:
        return 1 if self.grid[x][y] == 'R' else 0

    def tabla3D(self) -> Tuple[int, int]:
        dp = [[[-1 for _ in range(self.max_steps + 1)] for _ in range(self.n)] for _ in range(self.n)]

        def f(x, y, t):
            if not self.valido(x, y):
                return -float('inf')
            if t == 0:
                return self.radaway(0, 0) if (x == 0 and y == 0) else -float('inf')
            if dp[x][y][t] != -1:
                return dp[x][y][t]
            max_radaway = -float('inf')
            for dx, dy in self.directions:
                nx, ny = x - dx, y - dy
                if self.valido(nx, ny):
                    prev = f(nx, ny, t - 1)
                    if prev != -float('inf'):
                        max_radaway = max(max_radaway, prev + self.radaway(x, y))
            dp[x][y][t] = max_radaway
            return max_radaway

        best = -float('inf')
        best_t = -1
        for t in range(self.max_steps + 1):
            val = f(self.n - 1, self.n - 1, t)
            if val > best:
                best = val
                best_t = t
        return max(0, best), best_t

    def tabla_hash(self) -> Tuple[int, int]:
        memo: Dict[Tuple[int, int, int], int] = {}

        def f(x, y, t):
            if not self.valido(x, y):
                return -float('inf')
            if t == 0:
                return self.radaway(0, 0) if (x == 0 and y == 0) else -float('inf')
            if (x, y, t) in memo:
                return memo[(x, y, t)]
            max_radaway = -float('inf')
            for dx, dy in self.directions:
                nx, ny = x - dx, y - dy
                if self.valido(nx, ny):
                    prev = f(nx, ny, t - 1)
                    if prev != -float('inf'):
                        max_radaway = max(max_radaway, prev + self.radaway(x, y))
            memo[(x, y, t)] = max_radaway
            return max_radaway

        best = -float('inf')
        best_t = -1
        for t in range(self.max_steps + 1):
            val = f(self.n - 1, self.n - 1, t)
            if val > best:
                best = val
                best_t = t
        return max(0, best), best_t

In [3]:
#cuadricula
def generar_random_grid(n: int, bomb_prob: float = 0.15, radaway_prob: float = 0.3) -> List[List[str]]:
    grid = [['.' for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if (i, j) in [(0, 0), (n-1, n-1)]:
                continue
            r = random.random()
            if r < bomb_prob:
                grid[i][j] = 'B'
            elif r < bomb_prob + radaway_prob:
                grid[i][j] = 'R'
    return grid
#rendimiento
def rendimiento(refugio, method_name, method_func):
    print(f"\n--- {method_name} ---")
    tracemalloc.start()
    start = time.time()
    result, steps = method_func()
    end = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print(f"Se recogieron {result} capsulas RadAway en {steps} pasos")
    print(f"Tiempo: {end - start:.6f} s, Memoria maxima usada: {peak / 1024 / 1024:.2f} MB")
    return {'result': result, 'steps': steps, 'time': end - start, 'mem_peak': peak}


In [5]:
def run_experiments():
    print("EXPERIMENTOS COMPARATIVOS")
    sizes = [10, 20, 30, 40]
    
    #para hacer los graficos
    caps_3d, caps_hash = [], []
    time_3d, time_hash = [], []
    mem_3d, mem_hash = [], []

    for n in sizes:
        print(f"\nTamaño n = {n}")
        random.seed(42 + n)
        grid = generar_random_grid(n)
        refugio = Refugio117(grid)

        res_3d = rendimiento(refugio, "Tabla 3D", refugio.tabla3D)
        res_hash = rendimiento(refugio, "Tabla Hash", refugio.tabla_hash)

        caps_3d.append(res_3d['result'])
        time_3d.append(res_3d['time'])
        mem_3d.append(res_3d['mem_peak'])

        caps_hash.append(res_hash['result'])
        time_hash.append(res_hash['time'])
        mem_hash.append(res_hash['mem_peak'])

        if res_3d['result'] != res_hash['result']:
            print("\nResultados distintos, revisar errores o aleatoriedad.")
        else:
            print("\nAmbos metodos coinciden con las capsulas RadAway recogidas.")

        speedup = res_3d['time'] / res_hash['time'] if res_hash['time'] else float('inf')
        mem_ratio = res_3d['mem_peak'] / res_hash['mem_peak'] if res_hash['mem_peak'] else float('inf')

        print(f"\nSpeedup tabla 3D/Hash: {speedup:.2f}x")
        print(f"Ratio memoria tabla 3D/Hash: {mem_ratio:.2f}x")

    return sizes, caps_3d, caps_hash, time_3d, time_hash, mem_3d, mem_hash
n_values, caps_3d, caps_hash, time_3d, time_hash, mem_3d, mem_hash = run_experiments()

EXPERIMENTOS COMPARATIVOS

Tamaño n = 10

--- Tabla 3D ---
Se recogieron 6 capsulas RadAway en 18 pasos
Tiempo: 0.007025 s, Memoria maxima usada: 0.20 MB

--- Tabla Hash ---
Se recogieron 6 capsulas RadAway en 18 pasos
Tiempo: 0.011527 s, Memoria maxima usada: 0.11 MB

Ambos metodos coinciden con las capsulas RadAway recogidas.

Speedup tabla 3D/Hash: 0.61x
Ratio memoria tabla 3D/Hash: 1.82x

Tamaño n = 20

--- Tabla 3D ---
Se recogieron 24 capsulas RadAway en 38 pasos
Tiempo: 0.057583 s, Memoria maxima usada: 0.30 MB

--- Tabla Hash ---
Se recogieron 24 capsulas RadAway en 38 pasos
Tiempo: 0.065191 s, Memoria maxima usada: 0.98 MB

Ambos metodos coinciden con las capsulas RadAway recogidas.

Speedup tabla 3D/Hash: 0.88x
Ratio memoria tabla 3D/Hash: 0.31x

Tamaño n = 30

--- Tabla 3D ---
Se recogieron 36 capsulas RadAway en 58 pasos
Tiempo: 0.193864 s, Memoria maxima usada: 1.16 MB

--- Tabla Hash ---
Se recogieron 36 capsulas RadAway en 58 pasos
Tiempo: 0.257982 s, Memoria maxima usad

Se ejecuto varias veces y con n=40 siempre nos da 0 capsulas  en -1 pasos, lo que indica que apesar de que se ejecute varias veces la configuracion de la cuadricula con sus bombas aleatorias 
hace que el objetivo de llegar a la otra punta de la cuadricula sea inalcanzable debido a las restricciones como el limite de pasos

si no se ven los graficos es por permisos.

In [6]:
# Cápsulas recolectadas
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=n_values, y=caps_3d, name="Tabla 3D", mode='lines+markers'))
fig1.add_trace(go.Scatter(x=n_values, y=caps_hash, name="Tabla Hash", mode='lines+markers'))
fig1.update_layout(title="Cápsulas RadAway recolectadas", xaxis_title="Tamaño n", yaxis_title="Cápsulas")
fig1.show()

# Tiempo
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=n_values, y=time_3d, name="Tabla 3D", mode='lines+markers'))
fig2.add_trace(go.Scatter(x=n_values, y=time_hash, name="Tabla Hash", mode='lines+markers'))
fig2.update_layout(title="Tiempo de ejecución", xaxis_title="Tamaño n", yaxis_title="Tiempo (s)")
fig2.show()

# Memoria
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=n_values, y=mem_3d, name="Tabla 3D", mode='lines+markers'))
fig3.add_trace(go.Scatter(x=n_values, y=mem_hash, name="Tabla Hash", mode='lines+markers'))
fig3.update_layout(title="Uso de memoria pico", xaxis_title="Tamaño n", yaxis_title="Memoria (MB)")
fig3.show()


El hecho de que con N=40 si se este usando memoria y si tenga tiempo de ejecucion como se ve en los graficos, indica que si esta intentado buscar la mejor ruta hacia (n-1,n-1) pero por las 
limitaciones preestablecidas como los movimientos maximos no logra llegar a (n-1,n-1) antes del limite de pasos